
### 1. Definition
熵：$H = -p(x)\sum\log_2p(x)$

条件熵：$H(Y|X)=\sum^n_{i=1}p_iH(Y|X=x_i)$

信息增益：$g(D, A) = H(D) - H(D|A)$（互信息)，其中，

In [1]:
import math

In [2]:
def getEntropy(data):
    size = len(data)
    label_count = {}
    for feature in data:
        cur_label = feature[-1]
        if cur_label not in label_count.keys():
            label_count[cur_label] = 0
        label_count[cur_label] += 1
    # print(label_count)
    entropy = 0
    for key in label_count:
        p = float(label_count[key])/size
        # print("key: ", key, "prob: ",p)
        entropy -= p * math.log(p, 2)
    return entropy

In [3]:
data = [[1,1,'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
labels = ['no surfacing', 'flippers']
print(getEntropy(data))

0.9709505944546686


In [4]:
def splitData(data, axis, value):
    res = []
    for feature in data:
        if feature[axis] == value:
            reduced_feature = feature[:axis]
            reduced_feature.extend(feature[axis+1:])
            res.append(reduced_feature)
    return res

In [5]:
print(splitData(data, 0, 1))

[[1, 'yes'], [1, 'yes'], [0, 'no']]


In [6]:
# def conEntropy(data, axis, value): # 计算条件经验熵
#     for i in data:
         
            

In [7]:
'''
事实上，H(D)是一定的，根据信息增益公式：
g(D, A) = H(D) - H(D|A)
要求出max(g(D, A))
不需要计算信息增益，仅计算条件熵，并比较大小，选取条件熵较小的哪一个特征即可
To Do:
 - 修改代码，完成上述优化
'''
def chooseFeature(data):
    size = len(data[0]) - 1
    base = getEntropy(data)
    g_max = 0.0
    res = -1
    for i in range(size): # choose different axis as feature
        # print(i)
        feature_list = [example[i] for example in data]
        unique_val = set(feature_list)
        # print(unique_val)
        new_ent = 0
        for value in unique_val: # H = -plog(p) - (1-p)log(1-p)
            sub_data = splitData(data, i, value)
            p = len(sub_data)/float(len(data))
            new_ent += p * getEntropy(sub_data)
        g = base - new_ent
        if g > g_max:
            g_max = g
            res = i
    return res

In [8]:
print(chooseFeature(data))

0


In [9]:
def mostClass(class_list):
    return max(class_list, key = class_list.count)

In [10]:
l = [1,1,3,3,4,4,4,2]
print(mostClass(l))

4


In [11]:
def createTree(data, labels):
    class_list = [example[-1] for example in data]
    if class_list.count(class_list[0]) == len(class_list): # 只有一种类别
        return class_list[0]
    if len(data[0]) == 1: # 只有一个特征
        return mostClass(class_list)
    feature = chooseFeature(data)
    feature_label = labels[feature]
    my_tree = {feature_label: {}}
    del(labels[feature])
    feature_vals = [example[feature] for example in data]
    unique_vals = set(feature_vals)
    for val in unique_vals:
        sub_labels = labels[:]
        my_tree[feature_label][val] = createTree(splitData(data, feature, val), sub_labels)
    return my_tree

In [12]:
my_tree = createTree(data, labels)
print(my_tree)

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
